<a href="https://colab.research.google.com/github/SreyaSalil/IR-Assignments/blob/main/IR_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR Assignment 4

*Implementation of Probablistic model*

## Import Packages

In [26]:
import string
import itertools
import math
import operator
#For acessing files
import glob 
import errno
#To remove Numbers in text using RE
import re
#accent removal
import unicodedata
#stop word removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
#Stemming
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Retrieve Documents

In [27]:
path = '/content/sample_data/*.txt' 
files = glob.glob(path)
print(files)

['/content/sample_data/T5.txt', '/content/sample_data/T4.txt', '/content/sample_data/T2.txt', '/content/sample_data/T7.txt', '/content/sample_data/T10.txt', '/content/sample_data/T3.txt', '/content/sample_data/T1.txt', '/content/sample_data/T8.txt', '/content/sample_data/T9.txt', '/content/sample_data/T6.txt']


## Document Preprocessing Functions

Read all lines in file

In [28]:
def readFile(file):
  line = file.read().replace("\n", " ")
  file.close()
  return line

Create a list of tuples (documentNumber, documentText)

In [29]:
uniqueTerms=set()
fileText=[]
i=0
for name in files:
  try: 
        with open(name,encoding="utf8", errors='ignore') as f:
          i=i+1
          text=readFile(f)
          fileText.append((i,text))
  except IOError as exc: 
        if exc.errno != errno.EISDIR: 
            raise

Lexical Analysis

In [30]:
def convertToLowercase(text):
  return text.lower()

def removeNumbers(text):
  return re.sub(r'\d+', '', text)

def removePunctuation(text):
  return text.translate(str.maketrans('','',string.punctuation))

def removeWhitespace(text):
  return ' '.join(text.split())

def lexicalAnalysis(text):
  text=convertToLowercase(text)
  text=removeNumbers(text)
  text=removePunctuation(text)
  text=removeWhitespace(text)
  return text

Accent Removal

In [31]:
def removeAccent(text):
  new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return new_text

Stop word elimination

In [32]:
def eliminateStopword(tokens):
  stop_words = set(stopwords.words('english'))
  result = [i for i in tokens if not i in stop_words and len(i) > 2]
  return result

Stemming

In [33]:
def stemming(tokens):
  stemmer= PorterStemmer()
  output=[stemmer.stem(word) for word in tokens]
  return output

Step-by-step document preprocessing

In [34]:
def preprocess_data(contents):
  dataDict = {}
  for content in contents:
    textLA=lexicalAnalysis(content[1])
    textRA=removeAccent(textLA)
    tokens=word_tokenize(textRA)
    tokenES=eliminateStopword(tokens)
    stems=stemming(tokenES)
    finalTokens=eliminateStopword(stems)
    dataDict[content[0]] = finalTokens
  return dataDict

Query Preprocessing

In [35]:
def preprocess_query(query):
    textLA=lexicalAnalysis(query)
    textRA=removeAccent(textLA)
    tokens=word_tokenize(textRA)
    tokenES=eliminateStopword(tokens)
    stems=stemming(tokenES)
    return stems

## Generate Inverted Index

Functions to generate inverted index

In [36]:
def get_vocabulary(data):
    tokens = []
    for token_list in data.values():
        tokens = tokens + token_list
    fdist = nltk.FreqDist(tokens)
    return list(fdist.keys())

In [37]:
def generate_inverted_index(data):
    all_words = get_vocabulary(data)
    index = {}
    for word in all_words:
        for doc, tokens in data.items():
            if word in tokens :
                if word in index.keys():
                    index[word].append(doc)
                else:
                    index[word] = [doc]
    return index

## Preprocessing Query and Documents

Preprocessing documents and generating inverted index

In [38]:
preprocessed_data = preprocess_data(fileText)
inverted_index = generate_inverted_index(preprocessed_data)
inverted_index

{'birth': [1, 2, 4, 7, 10],
 'death': [1, 4, 7, 10],
 'star': [1, 2, 3, 5, 6, 7, 9, 10],
 'variabl': [1, 4, 7],
 'new': [1, 3, 4, 5, 7, 8, 9, 10],
 'dark': [1, 2, 3, 6, 7, 8, 9],
 'die': [1, 3, 4, 10],
 'sun': [1, 2, 3, 5, 6, 7, 9, 10],
 'mani': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'astronom': [1, 2, 3, 5, 6, 7],
 'believ': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'anoth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'follow': [1, 2, 3, 4, 5, 7, 8, 10],
 'dullest': [1],
 'red': [1, 2, 3, 4, 6, 7, 8, 9],
 'light': [1, 2, 3, 4, 5, 6, 7, 9, 10],
 'vari': [1, 4, 5, 7, 9, 10],
 'period': [1, 3, 4, 5, 6, 7, 9, 10],
 'day': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'week': [1, 4],
 'year': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'interest': [1, 2, 4, 5, 6, 7, 8, 9, 10],
 'specul': [1, 2, 6, 7, 9, 10],
 'slowli': [1, 2, 3, 4, 5, 6, 9, 10],
 'molten': [1, 2, 10],
 'interior': [1, 2, 3, 6, 9],
 'burst': [1, 2, 6],
 'shell': [1, 4, 7, 8, 9, 10],
 'thick': [1, 2, 3, 4, 7, 10],
 'vapour': [1, 2, 3, 6, 7],
 'gather': [1, 8, 9, 10],
 

Accepting and Preprocessing Query

In [39]:
inputQuery = input('Enter the query:')
preprocessed_query = preprocess_query(inputQuery)
preprocessed_query

Enter the query:Fish zeta limit


['fish', 'zeta', 'limit']

## Calculate Nw and (N-Nw)/Nw values

In [40]:
def cal_nw(inverted_index):
    matrix={}
    for token in inverted_index.keys():
        termInvIndex=inverted_index[token]
        matrix[token]=len(termInvIndex)
    return matrix

matrix=cal_nw(inverted_index)
matrix

{'birth': 5,
 'death': 4,
 'star': 8,
 'variabl': 3,
 'new': 8,
 'dark': 7,
 'die': 4,
 'sun': 8,
 'mani': 10,
 'astronom': 6,
 'believ': 10,
 'anoth': 10,
 'follow': 8,
 'dullest': 1,
 'red': 8,
 'light': 9,
 'vari': 6,
 'period': 8,
 'day': 10,
 'week': 2,
 'year': 10,
 'interest': 9,
 'specul': 6,
 'slowli': 8,
 'molten': 3,
 'interior': 5,
 'burst': 3,
 'shell': 6,
 'thick': 6,
 'vapour': 5,
 'gather': 4,
 'round': 10,
 'saw': 4,
 'seem': 10,
 'point': 10,
 'stage': 6,
 'futur': 7,
 'howev': 9,
 'receiv': 7,
 'opinion': 4,
 'may': 10,
 'pass': 9,
 'great': 10,
 'swarm': 4,
 'meteor': 4,
 'region': 7,
 'space': 8,
 'rich': 5,
 'cosmic': 3,
 'dust': 5,
 'sort': 8,
 'cours': 9,
 'illumin': 4,
 'would': 10,
 'take': 10,
 'place': 9,
 'one': 10,
 'class': 6,
 'name': 8,
 'algol': 1,
 'special': 4,
 'everi': 9,
 'third': 6,
 'night': 6,
 'reduc': 4,
 'sever': 7,
 'hour': 8,
 'modern': 6,
 'astronomi': 4,
 'discov': 8,
 'case': 8,
 'realli': 5,
 'two': 10,
 'circul': 3,
 'common': 6,
 'ce

In [41]:
def wordProbablities(matrix):
  for token in matrix.keys():
    nw=matrix[token]
    prob=(len(files)-nw+0.5)/(nw+0.5)
    matrix[token]=[nw,prob]
  return matrix

wordProbablities(matrix)


{'birth': [5, 1.0],
 'death': [4, 1.4444444444444444],
 'star': [8, 0.29411764705882354],
 'variabl': [3, 2.142857142857143],
 'new': [8, 0.29411764705882354],
 'dark': [7, 0.4666666666666667],
 'die': [4, 1.4444444444444444],
 'sun': [8, 0.29411764705882354],
 'mani': [10, 0.047619047619047616],
 'astronom': [6, 0.6923076923076923],
 'believ': [10, 0.047619047619047616],
 'anoth': [10, 0.047619047619047616],
 'follow': [8, 0.29411764705882354],
 'dullest': [1, 6.333333333333333],
 'red': [8, 0.29411764705882354],
 'light': [9, 0.15789473684210525],
 'vari': [6, 0.6923076923076923],
 'period': [8, 0.29411764705882354],
 'day': [10, 0.047619047619047616],
 'week': [2, 3.4],
 'year': [10, 0.047619047619047616],
 'interest': [9, 0.15789473684210525],
 'specul': [6, 0.6923076923076923],
 'slowli': [8, 0.29411764705882354],
 'molten': [3, 2.142857142857143],
 'interior': [5, 1.0],
 'burst': [3, 2.142857142857143],
 'shell': [6, 0.6923076923076923],
 'thick': [6, 0.6923076923076923],
 'vapou

## Get Document Probablities

In [42]:
def getDocProb(matrix,inverted_index,files,preprocessed_query):
  DocProb={}
  for i in range(1,len(files)+1):
    prob=1
    for token in preprocessed_query:
      if i in inverted_index[token]:
        matrixData=matrix[token]
        prob=prob*matrixData[1]
      if prob==1:
        DocProb[i]=0
      else:
        DocProb[i]=prob
  return DocProb

## Output: Most relevant documents based on given query

In [43]:
probs=getDocProb(matrix,inverted_index,files,preprocessed_query)
list(sorted(probs.items(), key=lambda item: item[1],reverse=True))

[(2, 1.4444444444444444),
 (7, 1.4444444444444444),
 (4, 0.42483660130718953),
 (10, 0.42483660130718953),
 (5, 0.29411764705882354),
 (6, 0.29411764705882354),
 (8, 0.29411764705882354),
 (9, 0.29411764705882354),
 (1, 0),
 (3, 0)]